# REMOTE SENSING AND WATER MASS CLASSIFICATION
## BY Caroline O'Hagan
### Ecosystem Dynamics and Assessment Branch of the Northeast Fisheries Science Center
#### Narragansett, Rhode Island

### BACKGROUND

<u> “2023 was a crazy year”
* There was a very large and very unusual phytoplankton bloom in the Gulf of Maine
* This bloom lasted from April to August and caused reduction in water clarity, impacted harpoon fishing, and affected visual predators
* One hypothesis for reasoning this bloom occurred was there was an introduction of scotian shelf water coming into the region but there's no final answer
* Fisheries managers are really interested in following up with this
* The Gulf Stream also shifted North
* The northwardward shift resulted in warm and salty surface waters onto the shelf itself and inhibited warm rings
* This was problamatic because the species in this region were being affected due to not being adapted to the warmer saltier waters

<u> Orienting the Northeast Shelf
* The northeast shelf is a highly dynamic and highly productive region which makes it a very important study area for fisheries as a whole and in our situation where we’re looking at data for a highly unusual phytoplankton bloom
* The location of Georges Bank, the Gulf of Maine, the Scotian Shelf, and the Mid Atlantic Bight all are located and how their currents flow is  important for us to know in terms of where, what, and why water is present as well as what we expect for water masses
* We also track warm and cold core rings and where they interact with water
* Scotian Shelf water is cooler shallow water
* Warm-core rings are rotating warm masses of water surrounded by colder water
* Definitions:
* Water mass: a body water that originates in a specific regio
* Water characteristics: the biophysical parameters such as temperature, salinity, and phytoplankton biomass
* Water class: the model output identifying the type of water based on its characteristi

### What is this Seascapes?

<u> Seascapes:
* A model product
* Hosted on the CoastWatch website
* A current water mass classification product
* Global
* Clustering techniques into 33 water classes
* Variables model is built on indlude, chlorophyll, PAR, and SST
* Has potential uses in habitat biodiversity and ecosystem monitoring

### INTRODUCTION

<u> The overall project goal:
* Develop a fisheries relevant water mass classification product for the Northeast U.S. continental shelf ecosyste

<u> My project:
* Focused on assessing Seascapes, a global water mass classification product
* Data from the year 2023
* An important component of the larger project goal to refine, regionally tune, and validate the global Seascapes product to better represent water masses observed in the Northwest Atlantic

### METHODS

* The methods I used also highlight what I learned and the processes I had to go through
* Develop new Python code to download data from ERDDAP and other online satellite data repositories
* Analyze other remotely sensed variables such as chlorophyll concentration, sea surface temperature, and optical water type classifications to determine how they relate to the water classes identified by the Seascape modeL
* The satellite data was not all on the same map projection and resolution when downloaded
* For easier analysis and visualization, I re-gridded all of the satellite data products to the same map projection and resolution (SST and chlorphyll to match Seascapes)
* I analyzed multiple remotely sensed variables to better understand why Seascapes produced specific data, what the data meant, and how the data provided insights to the state of the ecosystem

### Loading in All Data

In [ ]:
import os #this is always what you have to start with or else nothing will be recognized

os.chdir(r'C:\users\caroline.OHagan\Downloads') #this is setting the directory

%run ./startup_file.ipynb #run startup file ALWAYS it is the magical code

%run ./FUNCTIONS.ipynb #the functions file is another that holds a lot of functions

start_date = '2023-01-01' #(yyyy-mm-dd format)
end_date = '2023-12-31' #this is the start and end dates that were worked with for my entire project

#SST data 
os.chdir(data_dir) #cd to data directory 
data_dir_fold = data_dir+'/SST' #path for new folder 
ds_source = downloaddata(start_date, end_date, 'sst')
#OCCCI Chlorophyll data
os.chdir(data_dir) #cd to data directory 
data_dir_fold = data_dir+'/Chlorophyll' #path for new folder 
dc_source = downloaddata(start_date, end_date, 'occci')
#Seascapes data 
os.chdir(data_dir) #cd to data directory 
data_dir_fold = data_dir+'/SEASCAPES' #path for new folder 
seascapes = downloaddata(start_date, end_date, 'seascapes')

#Process regridded SST data
eightday=ds_source.rolling(time=8, center=True, min_periods=1).mean() #create 8 day mean of sst data 
import xarray_regrid
ds_source = eightday.regrid.linear(seascapes) #regrid data 
ds_source= ds_source.where(
    ds_source.time==seascapes.time, drop=True) #only keeping dates that match the seascapes dates


num_files = len(seascapes.CLASS.values) #used for plotting

# CREATE DICTIONARIES OF MASKS
## masks were needed to mask out the NaN values
masks = {}
mask_one = []

for y in range(1,34):
    mask_one = []
    for x in range(num_files):
        cn= np.ma.masked_where(seascapes.CLASS.values[x]==y, seascapes.CLASS.values[x])
        mask_one.append(cn)
        masks.update({y: mask_one})

# masks were needed to mask out the NaN values

# CREATE DICTIONARIES OF PERCENTS
## percents calculated for seascapes plotting data
perce = []
percent = {}
for y in range(1,34):
    perce = []
    for x in range(num_files):
        #tt = masks[x][y] == y
        true_val = np.count_nonzero(masks[y][x].mask == True)
        #true_val = np.count_nonzero(tt == True)
        total_nonnan = np.count_nonzero(~np.isnan(masks[y][x].data)) #total number of pixels NON NAN
        perc = true_val / total_nonnan * 100  #PUT INTO LOOP AND FIND PERC FOR EACH DATE AND PLOT EACH DAT
        perce.append(perc)
        percent.update({y:perce})

#this turns the datetimes into strings and splits them into yyyy-mm-dd for labeling the date axis 
#add this BEFORE plot
d8s = []
month = []

for x in range(len(seascapes.time.data)):
    d88 = str(seascapes.time.data[x])
    d88 = d88.split('T')
    d8s.append(d88[0])
    M = seascapes.time[x].dt.strftime('%B').item()
    month.append(M)

### Cartopy Plot of Masked Values for ONE Seascapes Class

* One of the first steps to actually finalizing results for the Seascapes data was to determine when and where specific classes were identified
* 
We also assessed what classes where dominant and their persistence over tie
* 
We wanted to see if we could identify certain water massein our region
* Analyzing how well the water masses Seascapes identified matched with the masses we would actuallyexpectc

In [ ]:
#plot all dates on top of each other for ONE class
from matplotlib import colors as c
start_date = '2023-01-01' #(yyyy-mm-dd format)
end_date = '2023-12-31' #this is the start and end dates that were worked with for my entire project

cMap = c.ListedColormap(['w','b']) #define colormap (masked values will be white, presence will be in blue)

import numpy as np
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from shapely.ops import cascaded_union

mask = []
classnum = 14
for x in range(num_files):
    cn= np.ma.masked_where(seascapes.CLASS.values[x]==classnum, seascapes.CLASS.values[x])
    mask.append(cn)

for x in range(num_files):
    fig = plt.figure(figsize=(10, 7)) #set figure size
    map_projection = cartopy.crs.PlateCarree() #set map projection
    ax = plt.axes(projection=map_projection) 
    try:
        ax.pcolormesh(seascapes.longitude, seascapes.latitude, mask[x].mask, cmap=cMap) #set variables to plot
    except ValueError:
        pass
        
    ax.coastlines() #adds a line of the coast onto the plot
    bathym = cfeature.NaturalEarthFeature(name='bathymetry_K_200', scale='10m', category='physical') #bathymatry line
    bathym = cascaded_union(list(bathym.geometries()))
    ax.add_geometries(bathym, facecolor='none', edgecolor='black', crs=ccrs.PlateCarree())
    plt.title("Seascape Class 14" + ' '+seascapes.time[x].dt.strftime('%Y-%m-%d').item())

### Seascapes Stacked Bar Chart

In [ ]:
xi = list(range(len(seascapes.time))) #so we could force the x axis to plot all seascapes dates
## Will need to copy and paste this command in if seperately plotting plots down below

values = np.array([percent[1], percent[2], percent[3], percent[4],
                   percent[5], percent[6], percent[7], percent[8], percent[9], 
                   percent[10], percent[11], percent[12], percent[13], percent[14], 
                   percent[15], percent[16], percent[17], percent[18], percent[19], 
                   percent[20], percent[21], percent[22], percent[23], percent[24], 
                   percent[25], percent[26], percent[27], percent[28], percent[29], 
                   percent[30], percent[31], percent[32], percent[33]])

colors = ["midnightblue","navy","darkblue","mediumblue", "blue", "royalblue", 'cornflowerblue','steelblue','dodgerblue','deepskyblue','skyblue','lightskyblue','lightseagreen','mediumturquoise','mediumaquamarine','limegreen','lawngreen','greenyellow','yellow',
         'khaki','gold','goldenrod','darkgoldenrod','darkorange', 'chocolate','sandybrown','peru','coral','orangered','red','black','dimgrey', 'lightgray']


fig = plt.figure(figsize=(13, 11)) 
for i in range(values.shape[0]):

    p = plt.bar(xi, values[i], bottom = np.sum(values[:i], axis = 0), color = colors[i],label='Class' + ' ' + str(i+1)) #stacked bar chart
    plt.legend(bbox_to_anchor=(1.05, 1.0), ncol=2) #plot legend
    plt.xlabel("Date", fontsize=12) #give x axis a label
    plt.ylabel('Percent', fontsize=12) #give y axis a label
    plt.title('Percent Optical Seascapes Class', fontsize=14) #give plot a title
    plt.tick_params(axis='x', labelsize=10) #x axis tick parameters
    plt.xticks(xi, month, rotation=45, fontsize=10) #add correct labels
    plt.minorticks_on()
    plt.legend(reverse=True, loc='center left', bbox_to_anchor=(1, 0.5)) #call for legend to be able to be read bottom to top
    labels = [] #allowed me to only plot month title once instead of 4 times in a row
    for i in range(len(xi)): 
        if i % 4 == 0: 
            labels.append(month[i])
        else: 
            labels.append("")

    plt.xticks(ticks=xi,labels=labels) #gave the x axis the labels of month only being plotted once

* Seasonality is heavily shown in Seascapes identifying results
* Although the seasons changing definitely can have an effect on water masses, it shouldn’t completely shift the type of water that is present into a water class completely different
* We found that Seascapes characterizes water more so than identifies water masses

### Bar Plots of Seascapes Data

In [ ]:
#Each individual seascape class has its own bar plot here over the entire year
for i in range(1,34):
    plt.figure(figsize=(12, 4))
    p = plt.bar(xi, percent[i], color = colors[i-1],label='Class' + ' ' + str(i)) #set variables to plot 
    plt.xlabel("Date", fontsize=12)
    plt.ylabel('Percent', fontsize=12)
    plt.title('Percent Optical Seascapes Class', fontsize=14)
    plt.tick_params(axis='x', labelsize=10)
    plt.xticks(xi, month, rotation=45, fontsize=10) #add correct labels
    plt.ylim(0,60)
    #if values.shape[0]<5:
        #plt.bar(p)
    #else:
        #continue
    plt.minorticks_on()
    plt.legend(reverse=True, loc='center left', bbox_to_anchor=(1, 0.5))

### Cartopy Plot of Seascapes Data

In [ ]:
#This is a plot of the entire region we were focusing on this summer
#It is by date the Seascapes data was pulled from
#Wherever the correlating seascapes data appeared is where it shows up
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from shapely.ops import cascaded_union
for x in range(num_files):
    fig = plt.figure(figsize=(10, 7)) #set figure size
    map_projection = cartopy.crs.PlateCarree() #set map projection
    ax = plt.axes(projection=map_projection) 

    cm = LinearSegmentedColormap.from_list('my_colors', colors, N=33) #call colormap. these are the colors called from the stacked barchart
    im = ax.pcolormesh(seascapes.longitude, seascapes.latitude,seascapes.CLASS[x,:,:], cmap = cm, vmin=0, vmax=33) #set variables to plot
    ax.coastlines()
    bathym = cfeature.NaturalEarthFeature(name='bathymetry_K_200', scale='10m', category='physical')
    bathym = cascaded_union(list(bathym.geometries()))
    ax.add_geometries(bathym, facecolor='none', edgecolor='black', crs=ccrs.PlateCarree())

    ax.set_xticks(np.linspace(-78, -63.4, 5), crs=map_projection) #set latitude bounds 
    ax.set_yticks(np.linspace(34.3, 47, 5), crs=map_projection) #set longitude bounds
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)

    cb = plt.colorbar(im, label='Class number',) #colorbar title 
    tick_locator = ticker.MaxNLocator(nbins=33) #set 33 ticks to colorbar (one for each seascape class) 
    cb.locator = tick_locator
    cb.update_ticks()

### Colormap Matching the Exact Seascapes Identification Colors

In [ ]:
!pip install webcolors
import webcolors
new_cmap = [(0,0,152),(0,14,161),(0,29,169), (0,43,177), (0,58,186),(0,72,194),(0,86,201), (0,101,210), (0,115,218), (0,130,226), (0,144,235), (0,159,244), (16,177,217), (51,187,178), (85,197,137), (120,207,99), (154,217,60), (155,217,60), (189,226,21), (210,225,0), (215,211,0), (218,200, 0), (222, 189, 0), (227,187,0), (231,166,0), (234,154,0), (238,143,0), (243,130,0), (246,120,0), (251,108,0), (254,95,0), (50,50,50), (124,124,124), (200,200,200)]
colors = []
for x in range(1,34):
    cc= webcolors.rgb_to_hex(new_cmap[x])
    colors.append(cc)
xi = list(range(len(seascapes.time)))
values = np.array([percent[1], percent[2], percent[3], percent[4],
                   percent[5], percent[6], percent[7], percent[8], percent[9], 
                   percent[10], percent[11], percent[12], percent[13], percent[14], 
                   percent[15], percent[16], percent[17], percent[18], percent[19], 
                   percent[20], percent[21], percent[22], percent[23], percent[24], 
                   percent[25], percent[26], percent[27], percent[28], percent[29], 
                   percent[30], percent[31], percent[32], percent[33]])

fig = plt.figure(figsize=(13, 11)) 
for i in range(values.shape[0]):

    p = plt.bar(xi, values[i], bottom = np.sum(values[:i], axis = 0), color = colors[i],label='Class' + ' ' + str(i+1))
    plt.legend(bbox_to_anchor=(1.05, 1.0), ncol=2)
    plt.xlabel("Date", fontsize=12)
    plt.ylabel('Percent', fontsize=12)
    plt.title('Percent Optical Seascapes Class', fontsize=14)
    plt.tick_params(axis='x', labelsize=10)
    plt.xticks(xi, month, rotation=45, fontsize=10) #add correct labels
    plt.minorticks_on()
    plt.legend(reverse=True, loc='center left', bbox_to_anchor=(1, 0.5))
    labels = []
    for i in range(len(xi)): 
        if i % 4 == 0: 
            labels.append(month[i])
        else: 
            labels.append("")

    plt.xticks(ticks=xi,labels=labels)

### Subplots of SST, Chlorophyll, and Seascapes Data into One

In [ ]:
# Define the figure and each axis for the 3 rows and 3 columns
import numpy as np
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from shapely.ops import cascaded_union
colors = ["midnightblue","navy","darkblue","mediumblue", "blue", "royalblue", 'cornflowerblue','steelblue','dodgerblue','deepskyblue','skyblue','lightskyblue','lightseagreen','mediumturquoise','mediumaquamarine','limegreen','lawngreen','greenyellow','yellow',
         'khaki','gold','goldenrod','darkgoldenrod','darkorange', 'chocolate','sandybrown','peru','coral','orangered','red','black','dimgrey', 'lightgray']
cm = LinearSegmentedColormap.from_list('my_colors', colors, N=33) #call colormap

chloralog=np.log10(dc_source.chlor_a) #setting chlorophyll data into a log scale

for x in range(num_files):
    fig, axs = plt.subplots(nrows=1,ncols=3,
                            subplot_kw={'projection': ccrs.PlateCarree()},
                            figsize=(15,10))
    
    # axs is a 2 dimensional array of GeoAxes.  We will flatten it into a 1-D array
    axs=axs.flatten()
    bathym = cfeature.NaturalEarthFeature(name='bathymetry_K_200', scale='10m', category='physical')
    bathym = cascaded_union(list(bathym.geometries()))
    sstc=axs[0].pcolormesh(ds_source.longitude, ds_source.latitude,ds_source.sea_surface_temperature[x,:,:], cmap = 'jet') #set SST variables to plot
    axs[0].coastlines()
    axs[0].add_feature(cartopy.feature.LAND, zorder=100, facecolor='w')
    axs[0].add_geometries(bathym, facecolor='none', edgecolor='black', crs=ccrs.PlateCarree())
    axs[0].set_title('SST')
    plt.colorbar(sstc, ax=axs[0], shrink=0.35, label='SST (Celsius)') #colorbar title 
    chc=axs[1].pcolormesh(chloralog.longitude, chloralog.latitude, chlor[x,:,:], cmap = 'jet', vmin=-1, vmax=1) #set Chlorophyll variables to plot
    axs[1].coastlines()
    axs[1].add_feature(cartopy.feature.LAND, zorder=100, facecolor='w')
    axs[1].add_geometries(bathym, facecolor='none', edgecolor='black', crs=ccrs.PlateCarree())
    axs[1].set_title('Chlorophyll')
    cb= plt.colorbar(chc, ax=axs[1], shrink=0.35, label='Chlorophyll (mg/m^-3)')
    cb.set_ticks([-1.0, -0.52287875, 0.0, 0.47712125, 1.0])
    cb.set_ticklabels(['0.1', '0.3', '1', '3', '10'])
    sc=axs[2].pcolormesh(seascapes.longitude, seascapes.latitude,seascapes.CLASS[x,:,:], cmap = cm, vmin=0, vmax=33) #set Seascapes variables to plot
    axs[2].coastlines()
    axs[2].add_feature(cartopy.feature.LAND, zorder=100, facecolor='w') #masks out and masks land to be white so rivers and lakes aren't picked up on
    axs[2].add_geometries(bathym, facecolor='none', edgecolor='black', crs=ccrs.PlateCarree())
    axs[2].set_title('Seascapes')
    plt.colorbar(sc, ax=axs[2], shrink=0.35, label= 'Class Number') #colorbar title 
    plt.suptitle('Spring', fontsize=15, ha='right', va='center', x=0, y=0.45, rotation=90) #Title on the side of the plots
    fig.tight_layout() #so title doesn't have a huge space between it and plot

* The seasonality can be really seen in the subplots specficially in the mid July and late October plots. In these plots Seascapes is identifying water masses in July and putting them into classes and then come Cctober Seascapes identifies different water masses and puts them into classes
* We wouldn’t expect for there to be such a strong change in masses in this region just because of the seasons changing
* Also the fact that seascapes identifies different water masses in the Georges Bank and Gulf of Maine area is questionable because that water isn’t know to typically change
* The subplot of late October and the first Novemeber date are only either days apart. Seascapes identifies the shelf water masses as completely different water classes. This represents a huge inconsistency as there should not be such a shift in everything only 8 days apart
* The spring subplot is a positive example of something we would expect. In the SST plot there apears to be a warm core ring and in the Seascapes plot there is a ring where it identifies this water as tropical subtropical transition water which has a sea surface temperature average of 24.12 degrees Celsius and a chlorophyll average of 0.15mg/m^3, so everything aligns very nicely.
* Warm core ring has core of sargasso sea water which is exactly what is show

### SST Timeseries

In [ ]:
#Calculating averages so they can be plotted over time series
final_se = []
for x in range(1,34):
    se= stats.sem(averages[x])
    final_se.append(se)

#Average SST Timeseries by Seascapes Class
colors = ["midnightblue","navy","darkblue","mediumblue", "blue", "royalblue", 'cornflowerblue','steelblue','dodgerblue','deepskyblue','skyblue','lightskyblue','lightseagreen','mediumturquoise','mediumaquamarine','limegreen','lawngreen','greenyellow','yellow',
         'khaki','gold','goldenrod','darkgoldenrod','darkorange', 'chocolate','sandybrown','peru','coral','orangered','red','black','dimgrey', 'lightgray']


for x in range(1,34):
    fig = plt.figure(figsize=(12, 10)) #set figure size
    plt.scatter(xi, averages[x], label= 'Class' + ' ' + str(x), color = colors[x-1])
    plt.errorbar(xi,averages[x], yerr= std_err[x], color = 'dimgrey', ls='none')
    plt.title('Average SST by Seascape Class')
    plt.ylabel('Average SST (Celsius)')
    plt.xlabel('Date')
    plt.tick_params(axis='x', labelsize=10) #change date tick label size
    plt.xticks(xi, month, rotation=45, fontsize=8) #add correct labels
    plt.ylim(-1, 34)
    #plt.minorticks_on()
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))#add legend
    labels = []
    for i in range(len(xi)): 
        if i % 4 == 0: 
            labels.append(month[i])
        else: 
            labels.append("")

    plt.xticks(ticks=xi,labels=labels)
   
    for i in range(len(xi)):
        plt.annotate(f'{averages[x][i]:.2f}', (xi[i],averages[x][i]), fontsize=6, xycoords='data',
                xytext=(-10, 7), textcoords='offset points')


plt.show()

### Annual Average SST Data Plot

In [ ]:
stacked_bytime = ds_source.stack(z=('latitude', 'longitude')).mean(dim='time') #stacked all of the averages together into one
fig = plt.figure(figsize=(10, 7)) #set figure size
map_projection = cartopy.crs.PlateCarree() #set map projection
ax = plt.axes(projection=map_projection) 
plt.scatter(stacked_bytime.longitude, stacked_bytime.latitude,c= stacked_bytime.sea_surface_temperature, cmap = 'jet')
ax.coastlines()
ax.add_feature(cartopy.feature.LAND, zorder=100, facecolor='w') #whites out land
plt.title('Annual Average SST')
ax.set_xticks(np.linspace(-78, -63.4, 5), crs=map_projection) #set latitude bounds 
ax.set_yticks(np.linspace(34.3, 47, 5), crs=map_projection) #set longitude bounds
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)

### Chlorophyll Timeseries

In [ ]:
#Calculating averages so they can be plotted over time series
final_se = []
for x in range(1,34):
    se= stats.sem(averages[x])
    final_se.append(se)

#Average Chlorophyll Timeseries by Seascapes Class
colors = ["midnightblue","navy","darkblue","mediumblue", "blue", "royalblue", 'cornflowerblue','steelblue','dodgerblue','deepskyblue','skyblue','lightskyblue','lightseagreen','mediumturquoise','mediumaquamarine','limegreen','lawngreen','greenyellow','yellow',
         'khaki','gold','goldenrod','darkgoldenrod','darkorange', 'chocolate','sandybrown','peru','coral','orangered','red','black','dimgrey', 'lightgray']


for x in range(1,34):
    fig = plt.figure(figsize=(12, 5)) #set figure size
    for i in range(len(averages[x])):
        if averages[x][i]<0.01: #this is plotting the cholorphyll by data size
            a= plt.scatter(xi[i], averages[x][i], s= 5, label= "<0.01", color = colors[x])
        elif averages[x][i]>0.01 and averages[x][i]<0.1:
            b=plt.scatter(xi[i], averages[x][i], s= 10, label= ">0.01 and <0.1", color = colors[x])
        elif averages[x][i]>0.1 and averages[x][i]<0.5:
            c=plt.scatter(xi[i], averages[x][i], s= 25, label= ">0.1 and <0.5", color = colors[x])    
        elif averages[x][i]>0.5 and averages[x][i]<1:
            d=plt.scatter(xi[i], averages[x][i], s= 35, label= ">0.5 and <1", color = colors[x])  
        elif averages[x][i]>1 and averages[x][i]<1.5:
            e=plt.scatter(xi[i], averages[x][i], s= 45, label= ">1 and <1.5", color = colors[x]) 
        elif averages[x][i]>1.5 and averages[x][i]<2:
            f=plt.scatter(xi[i], averages[x][i], s= 55, label= ">1.5 and <2", color = colors[x])    
        elif averages[x][i]>2 and averages[x][i]<2.5:
            g=plt.scatter(xi[i], averages[x][i], s= 65, label= ">2 and <2.5", color = colors[x])
        elif averages[x][i]>2.5 and averages[x][i]<3:
            h=plt.scatter(xi[i], averages[x][i], s= 75, label= ">2.5 and <3", color = colors[x])   
        elif averages[x][i]>3 and averages[x][i]<3.5:
            i=plt.scatter(xi[i], averages[x][i], s= 85, label= ">3 and <3.5", color = colors[x])  
        elif averages[x][i]>3.5 and averages[x][i]<4:
            j=plt.scatter(xi[i], averages[x][i], s= 95, label= ">3.5 and <4", color = colors[x]) 
        elif averages[x][i]>4 and averages[x][i]<4.5:
            k=plt.scatter(xi[i], averages[x][i], s= 105, label= ">4 and <4.5", color = colors[x])
        elif averages[x][i]>4.5 and averages[x][i]<5:
            l=plt.scatter(xi[i], averages[x][i], s= 115, label= ">4.5 and <5", color = colors[x])
        elif averages[x][i]>5 and averages[x][i]<5.5:
            m=plt.scatter(xi[i], averages[x][i], s= 125, label= ">5 and <5.5", color = colors[x])
        elif averages[x][i]>5.5 and averages[x][i]<6:
            n=plt.scatter(xi[i], averages[x][i], s= 135, label= ">5.5 and <6", color = colors[x])
        elif averages[x][i]>6 and averages[x][i]<6.5:
            o=plt.scatter(xi[i], averages[x][i], s= 150, label= ">6 and <6.5", color = colors[x])
        elif averages[x][i]>6.5:
            p=plt.scatter(xi[i], averages[x][i], s= 200, label= ">6.5", color = colors[x])
    plt.title('Average CHLOROPHYLL by Seascape Class')
    plt.ylabel('Average CHLOR')
    plt.xlabel('Date')
    plt.tick_params(axis='x', labelsize=10) #change date tick label size
    plt.xticks(xi, month, rotation=45, fontsize=8) #add correct labels
    plt.ylim(0.05, 20)
    plt.yscale('symlog')
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys(), loc='center left', bbox_to_anchor=(1, 0.5), fontsize= '8.5', labelspacing = 1.5)
    labels = []
    for i in range(len(xi)): 
        if i % 4 == 0: 
            labels.append(month[i])
        else: 
            labels.append("")

    plt.xticks(ticks=xi,labels=labels)

    for i in range(len(xi)):
        plt.annotate(f'{averages[x][i]:.2f}', (xi[i],averages[x][i]), fontsize=6, xycoords='data',
                xytext=(-10, 7), textcoords='offset points')
        
    plt.show()

### Annual Average of Chlorophyll Data

In [ ]:
stacked_bytime = ds_source.stack(z=('latitude', 'longitude')).mean(dim='time')
fig = plt.figure(figsize=(10, 7)) #set figure size
map_projection = cartopy.crs.PlateCarree() #set map projection
ax = plt.axes(projection=map_projection) 
plt.scatter(stacked_bytime.longitude, stacked_bytime.latitude,c= stacked_bytime.chlor_a, vmin = 0, cmap= 'jet')
ax.coastlines()
plt.title('Average Chlorophyll')
ax.set_xticks(np.linspace(-78, -63.4, 5), crs=map_projection) #set latitude bounds 
ax.set_yticks(np.linspace(34.3, 47, 5), crs=map_projection) #set longitude bounds
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)

### CONCLUSION 

* By looking at the strong seasonality the Seascapes product produced over the year of 2023, as well observable inconsistencies, we can say that:
* Not a reliable product for quantifying water masses
* Seascapes determines water characteristics, however, this product needs to be tweaked so that it is correctly identifying water masses into it’s respectable classes
* Refining Seascapes to be a regionally tuned to the known Northeast shelf water masses
* This project was very exploratory so there are definitely some next steps to be taken. This includes, systematically looking for patterns and consistency in Seascapes performance, feature selection for the regional mode, and testing and validating the regional model

### This Matters!

* By enhancing remote sensing products and integrating multiple data sources, it contributes to a more comprehensive understanding of the Northeast U.S. continental shelf ecosystem, supporting sustainable fisheries management and habitat assessment
* Water mass classification are foundational for understanding the dynamics of oceanic environments, including nutrient distribution, plankton blooms, and thermal fronts. All of these components are critical factors influencing fisheries and their habitats
* The regionally tuned product will be used for the ecosystem based fisheries management and conservation efforts, as they help in monitoring habitat suitability, identifying potential fishing grounds, and predicting the movement of stocks

### CONCEPT MAP

Our concept map is a prime example of a complex system. We visually demonstrate a structure of multiple different components that have connections to each other in some shape or form and create a bigger picture and story